In [1]:
import sys
sys.path.append('../..')

import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from config import ATLAS_CONNECTION_STRING
from pymongo import MongoClient

In [2]:
client = MongoClient(ATLAS_CONNECTION_STRING)
osu_db = client['osu_random_db']

In [3]:
MIN_ACC, PP_RANGE, PP_INTERVAL, MAX_PP = .9, 200, 100, 10000
INTERVALS = math.ceil((MAX_PP - PP_RANGE) / PP_INTERVAL)

In [4]:
bm_popularity_pipeline = [
    {
        '$group': {
            '_id': '$beatmap_id',
            'count': {
                '$sum': 1
            }
        }
    },
    {
        '$sort': {
            'count': -1
        }
    }
]

In [5]:
def compute_acc(score):
    c50, c100, c300, cmiss = score['count50'], score['count100'], score['count300'], score['countmiss']
    return (c50 / 6 + c100 / 3 + c300) / (c50 + c100 + c300 + cmiss)

In [6]:
def criteria_curve(scores):
    est_pp = [s['mlpp']['est_user_pp'] for s in scores]
    acc = [compute_acc(s) for s in scores]

    criteria_pass = np.asarray(acc) > MIN_ACC
    cummulative_pass = np.cumsum(np.insert(criteria_pass, 0, False))

    total, n_pass = np.zeros(INTERVALS), np.zeros(INTERVALS)
    j_s, j_e = 0, 0

    for i in range(INTERVALS):
        b = i * PP_INTERVAL

        while j_s < len(est_pp) - 1 and est_pp[j_s] < b:
            j_s+=1
        
        while j_e < len(est_pp) - 1 and est_pp[j_e] < b + PP_RANGE:
            j_e+=1
        
        total[i] = j_e - j_s
        n_pass[i] = cummulative_pass[j_e] - cummulative_pass[j_s]
    
    return total, n_pass

In [7]:
def to_criteria_curve_doc(bm_id):

    fields = {
        'count50': 1,
        'count100': 1,
        'count300': 1,
        'countmiss': 1,
        'mlpp.est_user_pp': 1
    }

    def beatmapQuery(q): return list(
        osu_db['osu_scores_high'].find(
            {
                'beatmap_id': bm['_id'],
                **q
            },
            fields
        ).sort('mlpp.est_user_pp')
    )

    scores_nm = beatmapQuery({'enabled_mods': 0})
    scores_all = beatmapQuery({})

    total_nm, n_pass_nm = criteria_curve(scores_nm)
    total_unfiltered, n_pass_unfiltered = criteria_curve(scores_unfiltered)

    doc = {
        '_id': bm['_id'],
        'no_mod': {
            'total': total_nm.tolist(),
            'n_pass': n_pass_nm.tolist()
        },
        'all_scores': {
            'total': total_unfiltered.tolist(),
            'n_pass': n_pass_unfiltered.tolist()
        }
    }

    osu_db['beatmap_criteria_curve'].insert(doc)

In [8]:
top_bm = list(osu_db['osu_scores_high'].aggregate(bm_popularity_pipeline))

In [9]:
for bm in tqdm(top_bm[:3000]):
    to_criteria_curve_doc(bm['_id'])

  0%|          | 0/3000 [00:08<?, ?it/s]


NameError: name 'scores_unfiltered' is not defined

In [ ]:
total, n_pass = np.zeros(INTERVALS), np.zeros(INTERVALS)

for i in range(INTERVALS):
    b = i * PP_INTERVAL

    for j in range(len(est_pp)):
        if b < est_pp[j] < b + PP_RANGE:
            total[i] += 1
            n_pass[i] += criteria_pass[j]